In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from matplotlib.colors import SymLogNorm

from IPython.display import display
from lib.util_plot import *
from lib.constants import *
from lib.multilevel_analysis import *
from lib.pandas_impl import *
from lib.pandas_stats_VIS import *
from lib.matrix_analyzer import *
from lib.pandas_compute import *

import lib.VIS_L23_preprocessing.vis_L23_constants as VIS

In [ ]:
data_folder = Path.cwd() / 'data' / 'VIS'
eval_folder = Path.cwd() / 'data' / 'eval' / 'VIS_SBI_example_25-06-04'
eval_folder.mkdir(parents=True, exist_ok=True)
plot_folder = eval_folder / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
filename = eval_folder / "multilevel_analysis.pkl"
with open(filename, 'rb') as file:
    multilevel_analysis = pickle.load(file)

stats = multilevel_analysis.stats
df_summary = multilevel_analysis.df_summary

In [ ]:
df_foo = df_summary.copy().reset_index()

data_column = "model-P_preference"
exc_exc = df_foo[(df_foo.pre_celltype_merged == VIS.EXC_INH_UNKNOWN[0]) & (df_foo.post_celltype_merged == VIS.EXC_INH_UNKNOWN[0])][data_column].values
exc_inh = df_foo[(df_foo.pre_celltype_merged == VIS.EXC_INH_UNKNOWN[0]) & (df_foo.post_celltype_merged == VIS.EXC_INH_UNKNOWN[1])][data_column].values
inh_exc = df_foo[(df_foo.pre_celltype_merged == VIS.EXC_INH_UNKNOWN[1]) & (df_foo.post_celltype_merged == VIS.EXC_INH_UNKNOWN[0])][data_column].values
inh_inh = df_foo[(df_foo.pre_celltype_merged == VIS.EXC_INH_UNKNOWN[1]) & (df_foo.post_celltype_merged == VIS.EXC_INH_UNKNOWN[1])][data_column].values

In [ ]:
n_EE = exc_exc.size
n_zero_EE = n_EE - np.count_nonzero(exc_exc)
mean_EE = np.mean(exc_exc)
median_EE = np.median(exc_exc) 

n_EI = exc_inh.size
n_zero_EI = n_EI - np.count_nonzero(exc_inh)
mean_EI = np.mean(exc_inh)
median_EI = np.median(exc_inh)

n_IE = inh_exc.size
n_zero_IE = n_IE - np.count_nonzero(inh_exc)
mean_IE = np.mean(inh_exc)
median_IE = np.median(inh_exc)

n_II = inh_inh.size
n_zero_II = n_II - np.count_nonzero(inh_inh)
mean_II = np.mean(inh_inh)
median_II = np.median(inh_inh)

print("EE", n_EE, n_zero_EE/n_EE, mean_EE, median_EE)
print("EI", n_EI, n_zero_EI/n_EI, mean_EI, median_EI)
print("IE", n_IE, n_zero_IE/n_IE, mean_IE, median_IE)
print("II", n_II, n_zero_II/n_II, mean_II, median_II)

analytic_P = np.array([mean_EE, mean_EI, mean_IE, mean_II])

In [ ]:
params_inferred_P = np.loadtxt(eval_folder / "posterior_parameters_P")
params_inferred_P_mean = params_inferred_P.mean(axis=0)

params_inferred_PS = np.loadtxt(eval_folder / "posterior_parameters_PS")
params_inferred_PS_mean = params_inferred_PS.mean(axis=0)

params_inferred_PS_I = np.loadtxt(eval_folder / "posterior_parameters_PS_I")
params_inferred_PS_I_mean = params_inferred_PS_I.mean(axis=0)

params_inferred_PS_I_mean

In [ ]:
np.max(params_inferred_P)

In [ ]:
def get_normed(params, M):
    params = np.atleast_2d(params)
    params_normed = np.zeros_like(params)
    assert M.ndim == 1
    assert params.shape[1] == M.size
    for i in range(params.shape[0]):
        for j in range(params.shape[1]):
            if(params[i, j] <= 1):
                params_normed[i, j] = params[i,j] - 1
            else:
                params_normed[i, j] = (params[i, j]-1) / M[j]
    return params_normed

In [ ]:
M = np.array([5,5,5,5])
params_inferred_P_normed = get_normed(params_inferred_P, M)
params_analytic_P_normed = get_normed(analytic_P, M)

params_inferred_P_normed
params_analytic_P_normed

In [ ]:
data_column = "model-PS_preference"

exc_exc_S = df_foo[(df_foo.pre_celltype_merged == VIS.E) & (df_foo.post_celltype_merged == VIS.E) & (df_foo.post_compartment == VIS.SOMA[0])][data_column].values
exc_exc_D = df_foo[(df_foo.pre_celltype_merged == VIS.E) & (df_foo.post_celltype_merged == VIS.E) & (df_foo.post_compartment == VIS.DEND[0])][data_column].values
exc_exc_A = df_foo[(df_foo.pre_celltype_merged == VIS.E) & (df_foo.post_celltype_merged == VIS.E) & (df_foo.post_compartment == VIS.AIS[0])][data_column].values
exc_inh_S = df_foo[(df_foo.pre_celltype_merged == VIS.E) & (df_foo.post_celltype_merged == VIS.I) & (df_foo.post_compartment == VIS.SOMA[0])][data_column].values
exc_inh_D = df_foo[(df_foo.pre_celltype_merged == VIS.E) & (df_foo.post_celltype_merged == VIS.I) & (df_foo.post_compartment == VIS.DEND[0])][data_column].values
exc_inh_A = df_foo[(df_foo.pre_celltype_merged == VIS.E) & (df_foo.post_celltype_merged == VIS.I) & (df_foo.post_compartment == VIS.AIS[0])][data_column].values

params_analytic_PS = np.array([np.mean(exc_exc_S), np.mean(exc_exc_D), np.mean(exc_exc_A),
                                np.mean(exc_inh_S), np.mean(exc_inh_D), 1])

inh_exc_S = df_foo[(df_foo.pre_celltype_merged == VIS.I) & (df_foo.post_celltype_merged == VIS.E) & (df_foo.post_compartment == VIS.SOMA[0])][data_column].values
inh_exc_D = df_foo[(df_foo.pre_celltype_merged == VIS.I) & (df_foo.post_celltype_merged == VIS.E) & (df_foo.post_compartment == VIS.DEND[0])][data_column].values
inh_exc_A = df_foo[(df_foo.pre_celltype_merged == VIS.I) & (df_foo.post_celltype_merged == VIS.E) & (df_foo.post_compartment == VIS.AIS[0])][data_column].values
inh_inh_S = df_foo[(df_foo.pre_celltype_merged == VIS.I) & (df_foo.post_celltype_merged == VIS.I) & (df_foo.post_compartment == VIS.SOMA[0])][data_column].values
inh_inh_D = df_foo[(df_foo.pre_celltype_merged == VIS.I) & (df_foo.post_celltype_merged == VIS.I) & (df_foo.post_compartment == VIS.DEND[0])][data_column].values
inh_inh_A = df_foo[(df_foo.pre_celltype_merged == VIS.I) & (df_foo.post_celltype_merged == VIS.I) & (df_foo.post_compartment == VIS.AIS[0])][data_column].values

params_analytic_PS_I = np.array([np.mean(inh_exc_S), np.mean(inh_exc_D), np.mean(inh_exc_A),
                                np.mean(inh_inh_S), np.mean(inh_inh_D), 1])

In [ ]:
M = np.array([1, 1, 1, 1, 1, 1])
params_inferred_PS_normed = get_normed(params_inferred_PS, M)
params_analytic_PS_normed = get_normed(params_analytic_PS, M)

In [ ]:
M = np.array([10, 10, 15, 10, 10, 10])
params_inferred_PS_I_normed = get_normed(params_inferred_PS_I, M)
params_analytic_PS_I_normed = get_normed(params_analytic_PS_I, M)

In [ ]:
params_analytic_PS_I_normed

In [ ]:
def get_default_limits(n_params):
    limits = []
    for _ in range(n_params):
        limits.append([-1, 1])
    return limits

In [ ]:
alpha_labels_P = [r"$\alpha_{EE}$", r"$\alpha_{EI}$", r"$\alpha_{IE}$", r"$\alpha_{II}$"]
alpha_labels_PS = [r"$\alpha_{EE_S}$", r"$\alpha_{EE_D}$", r"$\alpha_{EE_A}$",r"$\alpha_{EI_S}$", r"$\alpha_{EI_D}$", r"$\alpha_{EI_A}$"]
alpha_labels_PS_I = [r"$\alpha_{IE_S}$", r"$\alpha_{IE_D}$", r"$\alpha_{IE_A}$",r"$\alpha_{II_S}$", r"$\alpha_{II_D}$", r"$\alpha_{II_A}$"]

### Posterior plots

In [ ]:
from sbi import analysis as analysis

fig, axes = analysis.pairplot(params_inferred_P_normed, limits=get_default_limits(4), figsize=(10,10), points=params_analytic_P_normed, labels=alpha_labels_P)
image = savefig_png_svg(fig, plot_folder / "posterior_parameters_P")    

In [ ]:
fig, axes = analysis.pairplot(params_inferred_PS_normed, limits=get_default_limits(6), figsize=(10,10), points=params_analytic_PS_normed, labels=alpha_labels_PS)
image = savefig_png_svg(fig, plot_folder / "posterior_parameters_PS")    

In [ ]:
fig, axes = analysis.pairplot(params_inferred_PS_I_normed, limits=get_default_limits(6), figsize=(10,10), points=params_analytic_PS_I_normed, labels=alpha_labels_PS_I)
image = savefig_png_svg(fig, plot_folder / "posterior_parameters_PS_I")
image

### Distribution plots

In [ ]:
import seaborn as sns
from IPython.display import display

from lib.util_plot import *

initPlotSettings(spines_top_right=True)

fig, axes = plt.subplots(2, 2, figsize=figsize_mm_to_inch(160, 50))

color1 = "grey"
color2 = "orange"
alpha = 0.3
num_bins = 40

binrange = (0, 10)

def get_nonzero_values(values): 
    return values


sns.histplot(get_nonzero_values(exc_exc), ax=axes[0, 0], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange, label="locally optimal specificity (analytic)")
sns.histplot(params_inferred_P[:,0], ax=axes[0, 0], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange, label="globally inferred specificity")
axes[0, 0].set_xlim(binrange)
axes[0, 0].set_ylabel("density")
axes[0, 0].set_title(r"$\alpha_{EE}$")
axes[0, 0].legend()
axes[0, 0].axvline(x=np.mean(exc_exc), color='grey', linestyle='-', linewidth=1.5)
axes[0, 0].axvline(x=params_inferred_P_mean[0], color='orange', linestyle='-', linewidth=1.5)
axes[0, 0].axvline(x=1, color='black', linestyle='-', linewidth=0.8)

sns.histplot(get_nonzero_values(exc_inh), ax=axes[0, 1], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_P[:,1], ax=axes[0, 1], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[0, 1].set_xlim(binrange)
axes[0, 1].set_ylabel(None)
axes[0, 1].set_title(r"$\alpha_{EI}$")
axes[0, 1].axvline(x=np.mean(exc_inh), color='grey', linestyle='-', linewidth=1.5)
axes[0, 1].axvline(x=params_inferred_P_mean[1], color='orange', linestyle='-', linewidth=1.5)
axes[0, 1].axvline(x=1, color='black', linestyle='-', linewidth=0.8)

sns.histplot(get_nonzero_values(inh_exc), ax=axes[1, 0], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_P[:,2], ax=axes[1, 0], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[1, 0].set_xlim(binrange)
axes[1, 0].set_ylabel("density")
axes[1, 0].set_title(r"$\alpha_{IE}$")
axes[1, 0].axvline(x=np.mean(inh_exc), color='grey', linestyle='-', linewidth=1.5)
axes[1, 0].axvline(x=params_inferred_P_mean[2], color='orange', linestyle='-', linewidth=1.5)
axes[1, 0].axvline(x=1, color='black', linestyle='-', linewidth=0.8)

sns.histplot(get_nonzero_values(inh_inh), ax=axes[1, 1], color=color1,  stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_P[:,3], ax=axes[1, 1], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[1, 1].set_xlim(binrange)
axes[1, 1].set_ylabel(None)
axes[1, 1].set_title(r"$\alpha_{II}$")
axes[1, 1].axvline(x=np.mean(inh_inh), color='grey', linestyle='-', linewidth=1.5)
axes[1, 1].axvline(x=params_inferred_P_mean[3], color='orange', linestyle='-', linewidth=1.5)
axes[1, 1].axvline(x=1, color='black', linestyle='-', linewidth=0.8)


plt.tight_layout()

img = savefig_png_svg(fig, plot_folder / f"parameter_distribution_P")

display(img)

In [ ]:
print(np.count_nonzero(exc_exc_S) / exc_exc_S.size)
print(np.count_nonzero(exc_exc_D) / exc_exc_D.size)
print(np.count_nonzero(exc_exc_A) / exc_exc_A.size)
print(np.count_nonzero(exc_inh_S) / exc_inh_S.size)
print(np.count_nonzero(exc_inh_D) / exc_inh_D.size)
#print(np.count_nonzero(exc_inh_A) / exc_inh_A.size)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=figsize_mm_to_inch(160, 50))


color1 = "grey"
color2 = "orange"
alpha = 0.3
num_bins = 20

binrange = (0, 2)

def get_nonzero_values(values): 
    return values


sns.histplot(get_nonzero_values(exc_exc_S), ax=axes[0, 0], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_PS[:,0], ax=axes[0, 0], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[0, 0].set_xlim(binrange)
axes[0, 0].set_ylabel("density")
axes[0, 0].set_title(r"$\alpha_{EE_S}$")
axes[0, 0].axvline(x=np.mean(exc_exc_S), color='grey', linestyle='-', linewidth=1.5)
axes[0, 0].axvline(x=params_inferred_PS_mean[0], color='orange', linestyle='-', linewidth=1.5)
axes[0, 0].axvline(x=1, color='black', linestyle='-', linewidth=0.8)


sns.histplot(get_nonzero_values(exc_exc_D), ax=axes[0, 1], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_PS[:,1], ax=axes[0, 1], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[0, 1].set_xlim(binrange)
axes[0, 1].set_ylabel(None)
axes[0, 1].set_title(r"$\alpha_{EE_D}$")
axes[0, 1].axvline(x=1, color='black', linestyle='-', linewidth=0.8)
axes[0, 1].axvline(x=np.mean(exc_exc_D), color='grey', linestyle='-', linewidth=1.5)
axes[0, 1].axvline(x=params_inferred_PS_mean[1], color='orange', linestyle='-', linewidth=0.8)


sns.histplot(get_nonzero_values(exc_exc_A), ax=axes[0, 2], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_PS[:,2], ax=axes[0, 2], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[0, 2].set_xlim(binrange)
axes[0, 2].set_ylabel(None)
axes[0, 2].set_title(r"$\alpha_{EE_A}$")
axes[0, 2].axvline(x=np.mean(exc_exc_A), color='grey', linestyle='-', linewidth=1.5)
axes[0, 2].axvline(x=params_inferred_PS_mean[2], color='orange', linestyle='-', linewidth=1.5)
axes[0, 2].axvline(x=1, color='black', linestyle='-', linewidth=0.8)


sns.histplot(get_nonzero_values(exc_inh_S), ax=axes[1, 0], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_PS[:,3], ax=axes[1, 0], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[1, 0].set_xlim(binrange)
axes[1, 0].set_ylabel("density")
axes[1, 0].set_title(r"$\alpha_{EI_S}$")
axes[1, 0].axvline(x=np.mean(exc_inh_S), color='grey', linestyle='-', linewidth=1.5)
axes[1, 0].axvline(x=params_inferred_PS_mean[3], color='orange', linestyle='-', linewidth=1.5)
axes[1, 0].axvline(x=1, color='black', linestyle='-', linewidth=0.8)


sns.histplot(get_nonzero_values(exc_inh_D), ax=axes[1, 1], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_PS[:,4], ax=axes[1, 1], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[1, 1].set_xlim(binrange)
axes[1, 1].set_ylabel(None)
axes[1, 1].set_title(r"$\alpha_{EI_D}$")
axes[1, 1].axvline(x=np.mean(exc_inh_D), color='grey', linestyle='-', linewidth=1.5)
axes[1, 1].axvline(x=params_inferred_PS_mean[4], color='orange', linestyle='-', linewidth=1.5)
axes[1, 1].axvline(x=1, color='black', linestyle='-', linewidth=0.8)


sns.histplot(get_nonzero_values(exc_inh_A), ax=axes[1, 2], color=color1, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
sns.histplot(params_inferred_PS[:,5], ax=axes[1, 2], color=color2, stat='density', alpha=alpha, bins=num_bins, binrange=binrange)
axes[1, 2].set_xlim(binrange)
axes[1, 2].set_ylabel(None)
axes[1, 2].set_ylabel("density")
axes[1, 2].set_title(r"$\alpha_{EI_A}$")
axes[1, 2].axvline(x=np.mean(exc_inh_A), color='grey', linestyle='-', linewidth=1.5)
axes[1, 2].axvline(x=params_inferred_PS_mean[5], color='orange', linestyle='-', linewidth=1.5)
axes[1, 2].axvline(x=1, color='black', linestyle='-', linewidth=0.8)

plt.tight_layout()

img = savefig_png_svg(fig, plot_folder / f"parameter_distribution_PS")

display(img)